In [1]:
#pyspark initialization
import findspark
findspark.init()

In [2]:
#importing libraries
import pyspark
import pyspark.sql.functions as sqlfunc
from pyspark.sql.functions import *

In [3]:
#building sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
#sample_data
sampleData = (("Ram", 28, "Sales", 3000),
              ("Meena", 33, "Sales", 4600),
              ("Robin", 40, "Sales", 4100),
              ("Kunal", 25, "Finance", 3000),
              ("Ram", 28, "Sales", 3000),
              ("Srishti", 46, "Management", 3300),
              ("Jeny", 26, "Finance", 3900),
              ("Hitesh", 30, "Marketing", 3000),
              ("Kailash", 29, "Marketing", 2000),
              ("Sharad", 39, "Sales", 4100)
              )
columns = ["Employee_Name", "Age", "Department", "Salary"]
df = spark.createDataFrame(data=sampleData, schema=columns)
df.printSchema()
df.show()

root
 |-- Employee_Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+-------------+---+----------+------+
|Employee_Name|Age|Department|Salary|
+-------------+---+----------+------+
|          Ram| 28|     Sales|  3000|
|        Meena| 33|     Sales|  4600|
|        Robin| 40|     Sales|  4100|
|        Kunal| 25|   Finance|  3000|
|          Ram| 28|     Sales|  3000|
|      Srishti| 46|Management|  3300|
|         Jeny| 26|   Finance|  3900|
|       Hitesh| 30| Marketing|  3000|
|      Kailash| 29| Marketing|  2000|
|       Sharad| 39|     Sales|  4100|
+-------------+---+----------+------+



In [5]:
# defining window partition
from pyspark.sql.window import Window
windowPartition = Window.partitionBy("Department").orderBy("Age")

# Analytical Window Functions

In [6]:
#analytical window function
# cume_dist() window function is used to get the cumulative distribution within a window partition
from pyspark.sql.functions import cume_dist
df.withColumn("cume_dist", cume_dist().over(windowPartition)).show()

+-------------+---+----------+------+---------+
|Employee_Name|Age|Department|Salary|cume_dist|
+-------------+---+----------+------+---------+
|        Kunal| 25|   Finance|  3000|      0.5|
|         Jeny| 26|   Finance|  3900|      1.0|
|      Srishti| 46|Management|  3300|      1.0|
|      Kailash| 29| Marketing|  2000|      0.5|
|       Hitesh| 30| Marketing|  3000|      1.0|
|          Ram| 28|     Sales|  3000|      0.4|
|          Ram| 28|     Sales|  3000|      0.4|
|        Meena| 33|     Sales|  4600|      0.6|
|       Sharad| 39|     Sales|  4100|      0.8|
|        Robin| 40|     Sales|  4100|      1.0|
+-------------+---+----------+------+---------+



In [7]:
#analytical window function
# lag() function: access previous rows’ data as per the defined offset value in the function
from pyspark.sql.functions import lag
df.withColumn('lag', lag('Salary', 1).over(windowPartition)).show()

+-------------+---+----------+------+----+
|Employee_Name|Age|Department|Salary| lag|
+-------------+---+----------+------+----+
|        Kunal| 25|   Finance|  3000|NULL|
|         Jeny| 26|   Finance|  3900|3000|
|      Srishti| 46|Management|  3300|NULL|
|      Kailash| 29| Marketing|  2000|NULL|
|       Hitesh| 30| Marketing|  3000|2000|
|          Ram| 28|     Sales|  3000|NULL|
|          Ram| 28|     Sales|  3000|3000|
|        Meena| 33|     Sales|  4600|3000|
|       Sharad| 39|     Sales|  4100|4600|
|        Robin| 40|     Sales|  4100|4100|
+-------------+---+----------+------+----+



In [8]:
#analytical window function
# lead() function: access next rows data as per the defined offset value in the function
from pyspark.sql.functions import lead
df.withColumn("lead", lead("Salary", 1).over(windowPartition)).show()

+-------------+---+----------+------+----+
|Employee_Name|Age|Department|Salary|lead|
+-------------+---+----------+------+----+
|        Kunal| 25|   Finance|  3000|3900|
|         Jeny| 26|   Finance|  3900|NULL|
|      Srishti| 46|Management|  3300|NULL|
|      Kailash| 29| Marketing|  2000|3000|
|       Hitesh| 30| Marketing|  3000|NULL|
|          Ram| 28|     Sales|  3000|3000|
|          Ram| 28|     Sales|  3000|4600|
|        Meena| 33|     Sales|  4600|4100|
|       Sharad| 39|     Sales|  4100|4100|
|        Robin| 40|     Sales|  4100|NULL|
+-------------+---+----------+------+----+



# Ranking Window Functions

In [9]:
#sample_data
sampleData = ((101, "Ram", "Biology", 80),
              (103, "Meena", "Social Science", 78),
              (104, "Robin", "Sanskrit", 58),
              (102, "Kunal", "Physics", 89),
              (101, "Ram", "Biology", 80),
              (106, "Hitesh", "Maths", 88),
              (108, "Jeny", "Physics", 75),
              (107, "Hitesh", "Maths", 88),
              (109, "Kailash", "Maths", 90),
              (105, "Sharad", "Social Science", 84)
              )
columns = ["Roll_No", "Student_Name", "Subject", "Marks"]
df2 = spark.createDataFrame(data=sampleData, schema=columns)
df2.printSchema()
df2.show()

root
 |-- Roll_No: long (nullable = true)
 |-- Student_Name: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Marks: long (nullable = true)

+-------+------------+--------------+-----+
|Roll_No|Student_Name|       Subject|Marks|
+-------+------------+--------------+-----+
|    101|         Ram|       Biology|   80|
|    103|       Meena|Social Science|   78|
|    104|       Robin|      Sanskrit|   58|
|    102|       Kunal|       Physics|   89|
|    101|         Ram|       Biology|   80|
|    106|      Hitesh|         Maths|   88|
|    108|        Jeny|       Physics|   75|
|    107|      Hitesh|         Maths|   88|
|    109|     Kailash|         Maths|   90|
|    105|      Sharad|Social Science|   84|
+-------+------------+--------------+-----+



In [10]:
# defining window partition
from pyspark.sql.window import Window
windowPartition1 = Window.partitionBy("Subject").orderBy("Marks")

In [11]:
#ranking window function
# row_number(): used to give a sequential number to each row present in the table
from pyspark.sql.functions import row_number
df2.withColumn("row_number", row_number().over(windowPartition1)).show()

+-------+------------+--------------+-----+----------+
|Roll_No|Student_Name|       Subject|Marks|row_number|
+-------+------------+--------------+-----+----------+
|    101|         Ram|       Biology|   80|         1|
|    101|         Ram|       Biology|   80|         2|
|    106|      Hitesh|         Maths|   88|         1|
|    107|      Hitesh|         Maths|   88|         2|
|    109|     Kailash|         Maths|   90|         3|
|    108|        Jeny|       Physics|   75|         1|
|    102|       Kunal|       Physics|   89|         2|
|    104|       Robin|      Sanskrit|   58|         1|
|    103|       Meena|Social Science|   78|         1|
|    105|      Sharad|Social Science|   84|         2|
+-------+------------+--------------+-----+----------+



In [12]:
#ranking window function
# rank(): used to give ranks to rows specified in the window partition
# leaves gaps in rank if there are ties
from pyspark.sql.functions import rank
df2.withColumn("rank", rank().over(windowPartition1)).show()

+-------+------------+--------------+-----+----+
|Roll_No|Student_Name|       Subject|Marks|rank|
+-------+------------+--------------+-----+----+
|    101|         Ram|       Biology|   80|   1|
|    101|         Ram|       Biology|   80|   1|
|    106|      Hitesh|         Maths|   88|   1|
|    107|      Hitesh|         Maths|   88|   1|
|    109|     Kailash|         Maths|   90|   3|
|    108|        Jeny|       Physics|   75|   1|
|    102|       Kunal|       Physics|   89|   2|
|    104|       Robin|      Sanskrit|   58|   1|
|    103|       Meena|Social Science|   78|   1|
|    105|      Sharad|Social Science|   84|   2|
+-------+------------+--------------+-----+----+



In [13]:
#ranking window function
#percent_rank(): provides rank to rows in a percentile format
from pyspark.sql.functions import percent_rank
df2.withColumn("percent_rank", percent_rank().over(windowPartition1)).show()

+-------+------------+--------------+-----+------------+
|Roll_No|Student_Name|       Subject|Marks|percent_rank|
+-------+------------+--------------+-----+------------+
|    101|         Ram|       Biology|   80|         0.0|
|    101|         Ram|       Biology|   80|         0.0|
|    106|      Hitesh|         Maths|   88|         0.0|
|    107|      Hitesh|         Maths|   88|         0.0|
|    109|     Kailash|         Maths|   90|         1.0|
|    108|        Jeny|       Physics|   75|         0.0|
|    102|       Kunal|       Physics|   89|         1.0|
|    104|       Robin|      Sanskrit|   58|         0.0|
|    103|       Meena|Social Science|   78|         0.0|
|    105|      Sharad|Social Science|   84|         1.0|
+-------+------------+--------------+-----+------------+



In [14]:
#ranking window function
# dense_rank(): similar to rank() function, there is only one difference - dense_rank() function doesn't leave gaps in rank when there are ties
from pyspark.sql.functions import dense_rank
df2.withColumn("dense_rank", dense_rank().over(windowPartition1)).show()

+-------+------------+--------------+-----+----------+
|Roll_No|Student_Name|       Subject|Marks|dense_rank|
+-------+------------+--------------+-----+----------+
|    101|         Ram|       Biology|   80|         1|
|    101|         Ram|       Biology|   80|         1|
|    106|      Hitesh|         Maths|   88|         1|
|    107|      Hitesh|         Maths|   88|         1|
|    109|     Kailash|         Maths|   90|         2|
|    108|        Jeny|       Physics|   75|         1|
|    102|       Kunal|       Physics|   89|         2|
|    104|       Robin|      Sanskrit|   58|         1|
|    103|       Meena|Social Science|   78|         1|
|    105|      Sharad|Social Science|   84|         2|
+-------+------------+--------------+-----+----------+



# Aggregate Window Functions

In [15]:
#sample_data
sampleData = (("Ram", "Sales", 3000),
              ("Meena", "Sales", 4600),
              ("Robin", "Sales", 4100),
              ("Kunal", "Finance", 3000),
              ("Ram", "Sales", 3000),
              ("Srishti", "Management", 3300),
              ("Jeny", "Finance", 3900),
              ("Hitesh", "Marketing", 3000),
              ("Kailash", "Marketing", 2000),
              ("Sharad", "Sales", 4100)
              )
columns = ["Employee_Name", "Department", "Salary"]
df3 = spark.createDataFrame(data=sampleData, schema=columns)
df3.printSchema()
df3.show()

root
 |-- Employee_Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+-------------+----------+------+
|Employee_Name|Department|Salary|
+-------------+----------+------+
|          Ram|     Sales|  3000|
|        Meena|     Sales|  4600|
|        Robin|     Sales|  4100|
|        Kunal|   Finance|  3000|
|          Ram|     Sales|  3000|
|      Srishti|Management|  3300|
|         Jeny|   Finance|  3900|
|       Hitesh| Marketing|  3000|
|      Kailash| Marketing|  2000|
|       Sharad|     Sales|  4100|
+-------------+----------+------+



In [16]:
# defining window partition
from pyspark.sql.window import Window
windowPartitionAgg = Window.partitionBy("Department")

In [17]:
#aggregate window fundtions - avg, sum, min, max
from pyspark.sql.functions import col,avg,sum,min,max
df3.withColumn("Avg", avg(col("Salary")).over(windowPartitionAgg))\
.withColumn("Sum", sum(col("Salary")).over(windowPartitionAgg))\
.withColumn("Min", min(col("Salary")).over(windowPartitionAgg))\
.withColumn("Max", max(col("Salary")).over(windowPartitionAgg)).show()

+-------------+----------+------+------+-----+----+----+
|Employee_Name|Department|Salary|   Avg|  Sum| Min| Max|
+-------------+----------+------+------+-----+----+----+
|        Kunal|   Finance|  3000|3450.0| 6900|3000|3900|
|         Jeny|   Finance|  3900|3450.0| 6900|3000|3900|
|      Srishti|Management|  3300|3300.0| 3300|3300|3300|
|       Hitesh| Marketing|  3000|2500.0| 5000|2000|3000|
|      Kailash| Marketing|  2000|2500.0| 5000|2000|3000|
|          Ram|     Sales|  3000|3760.0|18800|3000|4600|
|        Meena|     Sales|  4600|3760.0|18800|3000|4600|
|        Robin|     Sales|  4100|3760.0|18800|3000|4600|
|          Ram|     Sales|  3000|3760.0|18800|3000|4600|
|       Sharad|     Sales|  4100|3760.0|18800|3000|4600|
+-------------+----------+------+------+-----+----+----+

